In [1]:
import tensorflow as tf
import numpy as np

# tf.enable_eager_execution()
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
impression_embed_size = 50 # 853,540개 
poi_embed_size = 50      # 13,352개
filter_embed_size = 50   # 208개
platform_embed_size = 5       # 55개
city_embed_size = 50          # 34,752개
sort_embed_size = 50 #interaction 빼고 one-hot
device_embed_size = 3 # mobile, desktop, tablet one-hot

rnn_input_size = impression_embed_size + 5 # action 5개로 통합 후 one-hot 

global_imp_idx=0
#LSTM
hidden_state_size = 128

In [3]:
# Parameters
learning_rate = 0.0001
seq_length = 15
num_epoch = 10
batch_size = 1

input_size = 100 # TBD

In [4]:
import pandas as pd
import functions as f
from collections import Counter

In [5]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
meta_df = pd.read_csv('./item_metadata.csv')
submit_df = pd.read_csv('./submission_popular.csv')

popular_df = f.get_popularity(train_df)

In [6]:
poi_idx = np.load('./npy/poi_names.npy', allow_pickle=True)
poi_idx = list(poi_idx)
impressions_idx = np.load('./npy/impressions_index.npy', allow_pickle=True)
impressions_idx = list(impressions_idx)
city_idx = np.load('./npy/city_names.npy', allow_pickle=True)
city_idx = list(city_idx)
platform_idx = np.load('./npy/platform_names.npy', allow_pickle=True)
platform_idx = list(platform_idx)
filter_idx = np.load('./npy/filter_merged.npy', allow_pickle=True)
filter_idx = list(filter_idx)
action_idx = np.load('./npy/action_type_names.npy', allow_pickle=True)
action_idx = list(action_idx)

sort_order_idx = np.load('./npy/sorting_names.npy', allow_pickle=True)
sort_order_idx = list(sort_order_idx)
device_idx = np.load('./npy/device_names.npy', allow_pickle=True)
device_idx = list(device_idx)
'''
poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])
'''

'\npoi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])\nimpression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])\ncity_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])\nplatform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])\nfilter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])\n'

In [7]:
train_label = train_df[(train_df["action_type"] == "clickout item")]     #train_label : (1,586,586, 12)
batch_indexes = np.random.permutation(len(train_label))

In [8]:
def embedding_function(domain, key):
    
    embedded_vector = tf.zeros([impression_embed_size])
    if domain == action_idx[9]:  #'poi'
        if key in poi_idx:
            idx = poi_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(poi_embedding, idx)
        
    elif (domain == action_idx[1])|(domain == action_idx[3])|(domain == action_idx[4])|(domain == action_idx[5])\
    |(domain == action_idx[6])|(domain == action_idx[8]):  #impression
        if key in impressions_idx:
            idx = impressions_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(impression_embedding, idx)
    
    elif domain == action_idx[7]: # search for destination, 'city'
        if key in city_idx:
            idx = city_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(city_embedding, idx)

    elif domain == 'platform':
        if key in platform_idx:
            idx = platform_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(platform_embedding, idx)
        
    elif domain ==action_idx[2]:  #'filter'
        if key in filter_idx:
            idx = filter_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(filter_embedding, idx)
        
    elif domain ==action_idx[0]:  #sorting
        embedded_vector = np.zeros([sort_embed_size])
        embedded_vector[sort_order_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    elif domain =='device':
        embedded_vector = np.zeros([device_embed_size])
#         embedded_vector[device_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    return embedded_vector

In [9]:
action_idx

['change of sort order',
 'clickout item',
 'filter selection',
 'interaction item deals',
 'interaction item image',
 'interaction item info',
 'interaction item rating',
 'search for destination',
 'search for item',
 'search for poi']

In [10]:
def action_embedding(action):
    if action == action_idx[0]:
        embed = tf.one_hot(0, 5)
    elif action == action_idx[2]:
        embed = tf.one_hot(1, 5)
    elif action == action_idx[7]:
        embed = tf.one_hot(2, 5)
    elif action == action_idx[9]:
        embed = tf.one_hot(3, 5)
    else:
        embed = tf.one_hot(4, 5)
    return embed

In [11]:
INDEX_user_id =0                                                 
INDEX_session_id =1                                            
INDEX_timestamp =2                                                
INDEX_step        =3                                                       
INDEX_action_type =4                                       
INDEX_reference   =5                                               
INDEX_platform    =6                                                
INDEX_city        =7                                          
INDEX_device      =8                                                 
INDEX_current_filters=9                                                
INDEX_impressions    =10   
INDEX_prices         =11    

In [12]:
def getSequenceData(act_list, ref_list, seq_length):
    embed_total = tf.zeros([rnn_input_size])  #추후 1 -> batch size 변경
    embed_total = tf.reshape(embed_total, [1, rnn_input_size])
    for step in range(np.shape(act_list)[1]): 
        embed_ref = embedding_function(act_list[step], ref_list[step])
        embed_action = action_embedding(act_list[step])
        
        embed_single_action = tf.concat((embed_action, embed_ref), axis=0)
        embed_total = tf.concat((embed_total, tf.reshape(embed_single_action, [1,55])), axis=0)
        
    return embed_total[1:,:]

In [13]:
def getActionList(user_label):
    step = user_label['step']
    if step > seq_length+1:
        step = seq_length+1
    label_point = user_label.name
    history_list = train_df[label_point-step+1:label_point]
    
    act_list=[]
    ref_list=[]
    for idx in range(len(history_list)):
        act_list.append(history_list.iloc[idx]['action_type'])
        ref_list.append(history_list.iloc[idx]['reference'])
    if len(act_list) < seq_length:
        act_list = np.concatenate((np.zeros([seq_length-len(act_list)]), act_list))
        ref_list = np.concatenate((np.zeros([seq_length-len(ref_list)]), ref_list))
    
    return np.reshape(act_list, [-1, seq_length]), np.reshape(ref_list, [-1, seq_length])

In [14]:
def recurrent_neural_network_model(Act_list, Ref_list):
    
    XX = getSequenceData(Act_list, Ref_list, seq_length)
    XX = tf.reshape(XX, [batch_size,seq_length, rnn_input_size])
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_state_size)    
    _ , states = tf.nn.dynamic_rnn(lstm_cell, XX, dtype=tf.float32)   
    
    return states

In [15]:
def dense_net(context, train_label):
     
    context = tf.reshape(context, [hidden_state_size])
        #platform
    
    if global_imp_idx in impressions_idx:
        idx = impressions_idx.index(global_imp_idx)
    else:
        idx = -1
    target_impression_embed = tf.nn.embedding_lookup(impression_embedding, idx)
    
        
    #platform_embed = embedding_function('platform', train_label[-1,INDEX_platform])
    key = train_label[-1,INDEX_platform]
    if key in platform_idx:
        idx = platform_idx.index(key)
    else:
        idx = 0
    platform_embed = tf.nn.embedding_lookup(platform_embedding, idx)
    #city, search for destination 해서 검색해봤던 city들
    #city_embed = embedding_function('city', train_label[-1,INDEX_city])
    key = train_label[-1,INDEX_city]
    if key in city_idx:
        idx = city_idx.index(key)
    else:
        idx = 0
    city_embed = tf.nn.embedding_lookup(city_embedding, idx)
    #device
    #device_embed = embedding_function('device', train_label[-1,INDEX_device])
    key = train_label[-1,INDEX_device]
    device_embed = np.zeros([device_embed_size])
#         embedded_vector[device_idx.index(key)] = 1
    device_embed= tf.convert_to_tensor(device_embed, dtype=tf.float32)
    #current_filter
    
    filter_b = train_label[-1,INDEX_current_filters]
    if pd.isna(filter_b):
        final_embed_current_filter = tf.zeros(filter_embed_size)
    else:
        final_embed_current_filter = tf.zeros(filter_embed_size)
#         filters_b = f.string_to_array(filter_b)
#         final_embed_current_filter = getWeightedAverage('filter', filters_b , filter_embed_size)
    
#     print("context", np.shape(context))
#     print("platform_embed", np.shape(platform_embed))
#     print("city_embed", np.shape(city_embed))
#     print("device_embed", np.shape(device_embed))
    
    XX_input= tf.concat((context, platform_embed, city_embed, device_embed, target_impression_embed),axis=0)
    
    XX_input = tf.reshape(XX_input, [-1, 236])
    layer_1 = tf.layers.dense(XX_input, 256,activation=tf.nn.relu)
    layer_2 = tf.layers.dense(layer_1, 128,activation=tf.nn.relu)
    layer_3 = tf.layers.dense(layer_2, 64, activation=tf.nn.relu)
    out_layer = tf.layers.dense(layer_3, 1)
    return out_layer

In [16]:
# Define the neural network
tf.reset_default_graph()
Act_list = tf.placeholder(tf.string, [None, seq_length])
Ref_list = tf.placeholder(tf.string, [None, seq_length])

Train_Label_df = tf.placeholder(tf.string, [None, 12])
Y_label = tf.placeholder(tf.float32, [None, 1])
Learing_Rate_ph = tf.placeholder(tf.float32)

poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
context = recurrent_neural_network_model(Act_list, Ref_list)
output = dense_net(context[0], Train_Label_df)
# loss = tf.losses.sigmoid_cross_entropy(Y_label, output)
# loss = tf.losses.mean_squared_error(Y_label, output)
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_label, logits=output)
train = tf.train.AdamOptimizer(learning_rate=Learing_Rate_ph).minimize(loss)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [18]:
sess = tf.Session()
saver = tf.train.Saver()

In [19]:
sess.run(tf.global_variables_initializer())

In [20]:
learning_rate = 0.00001

In [21]:
len(train_label)

1586586

In [22]:
label_df = None
for epoch in range(5):
    for k in range(len(train_label)):
        idx = batch_indexes[k*batch_size:(k+1)*batch_size]
        label_df = train_label.iloc[idx]
        try:
            for i in range(1):  # batch size
                user_label = label_df.iloc[i]
                train_label_batch = user_label["reference"]  # 클릭한 reference number
                train_candidates = user_label['impressions']
                train_candidates = f.string_to_array(train_candidates)   

                candidate_prices = user_label['prices']
                candidate_prices = f.string_to_array(candidate_prices)
                candidate_prices = np.array(candidate_prices).astype(int)

                label_index = train_candidates.index(train_label_batch)
                labels = np.zeros([len(train_candidates)],dtype=np.float32)
                labels[label_index]=1.0

                act_list, ref_list = getActionList(user_label)
                act_list = act_list.astype(str)
                ref_list = ref_list.astype(str)

                user_label = user_label.fillna('Nope')
                users = np.array(user_label)
                users= np.reshape(users,[-1,12])     
                users = users.astype(str)

                for h in range(len(train_candidates)):
                    global_imp_idx = np.reshape(int(train_candidates[h]))
                    print("global_imp_idx :", global_imp_idx)
                    if labels[h]==1:
                            _  = sess.run([train], feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                                         Y_label:np.reshape(labels[h],[-1,1]), \
                                                             Learing_Rate_ph: learning_rate*35.0})

                            if k%50==0:
                                loss_val = sess.run([loss], feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                                             Y_label:np.reshape(labels[h],[-1,1])})
                                print('epoch: ', epoch, 'step: ', k , "labels[h]==1 loss : ", loss_val)
                                saver.save(sess, './saves/1/my-model.ckpt', global_step=k)
                    else:
                            _, loss_val = sess.run([train, loss], feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                                         Y_label:np.reshape(labels[h],[-1,1]), \
                                                                            Learing_Rate_ph: learning_rate})
                            if k%50==0:
                                if h%10==0:
                                    print('epoch: ', epoch, 'step: ',k, "labels[h]==0 loss : ", loss_val)

        except:
            pass






KeyboardInterrupt: 

In [ ]:
learning_rate

In [ ]:
batch_size

In [ ]:
def getActionListTest(user_label):
    step = user_label['step']
    if step > seq_length+1:
        step = seq_length+1
    label_point = user_label.name
    history_list = test_df[label_point-step+1:label_point]
    
    act_list=[]
    ref_list=[]
    for idx in range(len(history_list)):
        act_list.append(history_list.iloc[idx]['action_type'])
        ref_list.append(history_list.iloc[idx]['reference'])
    if len(act_list) < seq_length:
        act_list = np.concatenate((np.zeros([seq_length-len(act_list)]), act_list))
        ref_list = np.concatenate((np.zeros([seq_length-len(ref_list)]), ref_list))
    
    return np.reshape(act_list, [-1, seq_length]), np.reshape(ref_list, [-1, seq_length])

In [ ]:
def test_TestSet(user_label):
    for i in range(1):  
#         user_label = label_df.iloc[i]
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   

        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)

        predictions = []

        act_list, ref_list = getActionListTest(user_label)
        act_list = act_list.astype(str)
        ref_list = ref_list.astype(str)
        
        user_label = user_label.fillna('Nope')
        users = np.array(user_label)
        users = users.astype(str)
        users= np.reshape(users,[-1,12])     

        for h in range(len(train_candidates)):
            prediction = sess.run(output, feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                                             Target_impression: np.reshape(int(train_candidates[h]),[-1,1])})
            predictions.append(prediction)

    return predictions

In [ ]:
def test_TrainSet(user_label):
    for i in range(1):  
#         user_label = label_df.iloc[i]
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   

        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)

        predictions = []

        act_list, ref_list = getActionList(user_label)
        act_list = act_list.astype(str)
        ref_list = ref_list.astype(str)
        
        user_label = user_label.fillna('Nope')
        users = np.array(user_label)
        users = users.astype(str)
        users= np.reshape(users,[-1,12])     

        for h in range(len(train_candidates)):
            prediction = sess.run(output, feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                                             Target_impression: np.reshape(int(train_candidates[h]),[-1,1])})
            predictions.append(prediction)

    return predictions

In [ ]:
test_TrainSet(train_label.iloc[0])

In [ ]:
k=0
i=0

In [ ]:
    idx = batch_indexes[k*batch_size:(k+1)*batch_size]
    label_df = train_label.iloc[idx]

In [ ]:
        user_label = label_df.iloc[i]
        train_label_batch = user_label["reference"]  # 클릭한 reference number
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)
        
        label_index = train_candidates.index(train_label_batch)
        labels = np.zeros([len(train_candidates)],dtype=int)
        labels[label_index]=1
        
        act_list, ref_list = getActionList(user_label)
        act_list = act_list.astype(str)
        ref_list = ref_list.astype(str)
        
        user_label = user_label.fillna('Nope')
        users = np.array(user_label)
        users= np.reshape(users,[-1,12])     
        users = users.astype(str)

In [ ]:
target_impression = np.reshape(int(train_candidates[0]),[-1,1])
if target_impression in impressions_idx:
    idx = impressions_idx.index(target_impression)
else:
    idx = -1
print('idx :', idx)
target_impression_embed = tf.nn.embedding_lookup(impression_embedding, idx)

In [ ]:
sess.run(target_impression_embed)

In [ ]:
prediction = sess.run(output, feed_dict={Act_list: act_list, Ref_list: ref_list, Train_Label_df: users,\
                                         Target_impression: np.reshape(int(train_candidates[0]),[-1,1])})

In [ ]:
prediction